In [1]:
library(ggplot2)
library(GGally)
library(lme4)

Loading required package: Matrix


In [2]:
path <- '/nfs/turbo/intmed-bnallamo-turbo/wsliu/Data/NRD/'

In [2]:
path <- '/nfs/turbo/umms-awaljee/wsliu/Data/NRD/'

In [72]:
train.df <- read.csv(file = paste0(path, 'cohorts/ami/comorb_train9.csv'))
tst.df <- read.csv(file = paste0(path, 'cohorts/ami/comorb_tst9.csv'))

In [4]:
comorbs <- c('CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF', 'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG', 'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG', 'CM_HTN_C', 'CM_HYPOTHY', 'CM_LIVER', 'CM_LYMPH', 'CM_LYTES', 'CM_METS', 'CM_NEURO', 'CM_OBESE', 'CM_PARA', 'CM_PERIVASC', 'CM_PSYCH', 'CM_PULMCIRC', 'CM_RENLFAIL', 'CM_TUMOR', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS')

In [73]:
train.df$HOSP_NRD <- as.factor(train.df$HOSP_NRD)
tst.df$HOSP_NRD <- as.factor(tst.df$HOSP_NRD)

train.df$FEMALE <- as.factor(train.df$FEMALE)
tst.df$FEMALE <- as.factor(tst.df$FEMALE)
for(c in comorbs){
    train.df[ , c] <- as.factor(train.df[ , c])
    tst.df[ , c] <- as.factor(tst.df[ , c])
}

train.df$readm30 <- as.numeric(train.df$readm30) - 1
tst.df$readm30 <- as.numeric(tst.df$readm30) - 1

In [9]:
head(train.df)

AGE,AWEEKEND,DIED,DISCWT,DISPUNIFORM,DMONTH,DQTR,DRG,DRGVER,DRG_NoPOA,⋯,CM_OBESE,CM_PARA,CM_PERIVASC,CM_PSYCH,CM_PULMCIRC,CM_RENLFAIL,CM_TUMOR,CM_ULCER,CM_VALVE,CM_WGHTLOSS
63,0,0,1.505411,1,11,4,247,32,247,⋯,0,0,0,0,0,0,0,0,0,0
84,0,0,2.359884,5,2,1,280,31,280,⋯,0,0,0,0,0,0,0,0,0,0
57,1,0,1.945870,6,5,2,280,31,280,⋯,1,0,0,0,0,1,0,0,0,0
55,0,0,3.319084,5,7,3,280,31,280,⋯,0,0,0,0,0,0,0,0,0,0
74,0,0,2.312238,1,2,1,247,31,247,⋯,0,0,0,0,0,0,0,0,0,0
61,1,0,1.962384,1,3,1,247,31,247,⋯,0,0,0,0,0,0,0,0,0,0


In [25]:
dim(train.df)
dim(tst.df)

[1] 163671    179

[1] 18186   179

In [74]:
result <- glmer(readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS + CM_CHF + CM_CHRNLUNG + CM_COAG + 
                CM_DEPRESS + CM_DM + CM_DMCX + CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES + CM_METS + 
                CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH + CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + 
                CM_VALVE + CM_WGHTLOSS + AGE + FEMALE + (1|HOSP_NRD), data = train.df, family = binomial, 
                control = glmerControl(optimizer = "nloptwrap", calc.derivs = FALSE), nAGQ = 0)

In [75]:
pred_score <- predict(result, newdata = tst.df, type = 'response')

In [76]:
pred_test <- data.frame(y_pred = pred_score, y_true = tst.df$readm30)

In [77]:
write.csv(pred_test, file = paste0(path, 'cohorts/ami/prediction_MELR9.csv'))

In [18]:
# results for nAGQ=10
print(result, corr=False)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS +  
    CM_CHF + CM_CHRNLUNG + CM_COAG + CM_DEPRESS + CM_DM + CM_DMCX +  
    CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES +  
    CM_METS + CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH +  
    CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + CM_VALVE +  
    CM_WGHTLOSS + AGE + FEMALE + (1 | HOSP_NRD)
   Data: train.df
      AIC       BIC    logLik  deviance  df.resid 
108132.52 108459.11 -54033.26 108066.52    146738 
Random effects:
 Groups   Name        Std.Dev.
 HOSP_NRD (Intercept) 0.2001  
Number of obs: 146771, groups:  HOSP_NRD, 874
Fixed Effects:
 (Intercept)      CM_AIDS1   CM_ALCOHOL1   CM_ANEMDEF1      CM_ARTH1  
    -3.44808       0.22609       0.06104       0.30234       0.17105  
 CM_BLDLOSS1       CM_CHF1  CM_CHRNLUNG1  

In [10]:
# results for nAGQ=2
print(result, corr=False)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 2) [glmerMod]
 Family: binomial  ( logit )
Formula: readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS +  
    CM_CHF + CM_CHRNLUNG + CM_COAG + CM_DEPRESS + CM_DM + CM_DMCX +  
    CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES +  
    CM_METS + CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH +  
    CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + CM_VALVE +  
    CM_WGHTLOSS + AGE + FEMALE + (1 | HOSP_NRD)
   Data: train.df
      AIC       BIC    logLik  deviance  df.resid 
115035.08 115365.26 -57484.54 114969.08    163638 
Random effects:
 Groups   Name        Std.Dev.
 HOSP_NRD (Intercept) 0.2044  
Number of obs: 163671, groups:  HOSP_NRD, 869
Fixed Effects:
 (Intercept)      CM_AIDS1   CM_ALCOHOL1   CM_ANEMDEF1      CM_ARTH1  
    -3.69359       0.11732       0.05313       0.30028       0.15956  
 CM_BLDLOSS1       CM_CHF1  CM_CHRNLUNG1   

In [14]:
# results for nAGQ=1
print(result, corr=False)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS +  
    CM_CHF + CM_CHRNLUNG + CM_COAG + CM_DEPRESS + CM_DM + CM_DMCX +  
    CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES +  
    CM_METS + CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH +  
    CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + CM_VALVE +  
    CM_WGHTLOSS + AGE + FEMALE + (1 | HOSP_NRD)
   Data: train.df
      AIC       BIC    logLik  deviance  df.resid 
108132.65 108459.24 -54033.33 108066.65    146738 
Random effects:
 Groups   Name        Std.Dev.
 HOSP_NRD (Intercept) 0.2     
Number of obs: 146771, groups:  HOSP_NRD, 874
Fixed Effects:
 (Intercept)      CM_AIDS1   CM_ALCOHOL1   CM_ANEMDEF1      CM_ARTH1  
    -3.44664       0.22608       0.06122       0.30194       0.17115  
 CM_BLDLOSS1       CM_CHF1  CM_CHRNLUNG1      CM_COAG1   CM_DEPRE

In [12]:
# results for nAGQ=0
print(result, corr=False)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 0) [glmerMod]
 Family: binomial  ( logit )
Formula: readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS +  
    CM_CHF + CM_CHRNLUNG + CM_COAG + CM_DEPRESS + CM_DM + CM_DMCX +  
    CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES +  
    CM_METS + CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH +  
    CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + CM_VALVE +  
    CM_WGHTLOSS + AGE + FEMALE + (1 | HOSP_NRD)
   Data: train.df
      AIC       BIC    logLik  deviance  df.resid 
108133.16 108459.75 -54033.58 108067.16    146738 
Random effects:
 Groups   Name        Std.Dev.
 HOSP_NRD (Intercept) 0.1995  
Number of obs: 146771, groups:  HOSP_NRD, 874
Fixed Effects:
 (Intercept)      CM_AIDS1   CM_ALCOHOL1   CM_ANEMDEF1      CM_ARTH1  
    -3.43712       0.22602       0.06116       0.30134       0.17114  
 CM_BLDLOSS1       CM_CHF1  CM_CHRNLUNG1   

## Cross Validation

In [4]:
comorbs <- c('CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF', 'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG', 'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG', 'CM_HTN_C', 'CM_HYPOTHY', 'CM_LIVER', 'CM_LYMPH', 'CM_LYTES', 'CM_METS', 'CM_NEURO', 'CM_OBESE', 'CM_PARA', 'CM_PERIVASC', 'CM_PSYCH', 'CM_PULMCIRC', 'CM_RENLFAIL', 'CM_TUMOR', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS')

In [8]:
cohort = 'pna'

In [9]:
for (tst_fold in 0:9){
    train.df <- read.csv(file = paste0(path, 'cohorts/', cohort, '/comorb_train', tst_fold, '.csv'))
    tst.df <- read.csv(file = paste0(path, 'cohorts/', cohort, '/comorb_tst', tst_fold, '.csv'))

    train.df$HOSP_NRD <- as.factor(train.df$HOSP_NRD)
    tst.df$HOSP_NRD <- as.factor(tst.df$HOSP_NRD)

    train.df$FEMALE <- as.factor(train.df$FEMALE)
    tst.df$FEMALE <- as.factor(tst.df$FEMALE)
    for(c in comorbs){
        train.df[ , c] <- as.factor(train.df[ , c])
        tst.df[ , c] <- as.factor(tst.df[ , c])
    }

    train.df$readm30 <- as.numeric(train.df$readm30) - 1
    tst.df$readm30 <- as.numeric(tst.df$readm30) - 1
    
    result <- glmer(readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS + CM_CHF + CM_CHRNLUNG + CM_COAG + 
                CM_DEPRESS + CM_DM + CM_DMCX + CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES + CM_METS + 
                CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH + CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + 
                CM_VALVE + CM_WGHTLOSS + AGE + FEMALE + (1|HOSP_NRD), data = train.df, family = binomial, 
                control = glmerControl(optimizer = "nloptwrap", calc.derivs = FALSE), nAGQ = 0)
    pred_score <- predict(result, newdata = tst.df, type = 'response')
    pred_test <- data.frame(y_pred = pred_score, y_true = tst.df$readm30)
    write.csv(pred_test, file = paste0(path, 'cohorts/', cohort, '/prediction_MELR', tst_fold, '.csv'))
    }